In [1]:
from arcgis.gis import GIS
from arcgis import features
from getpass import getpass #to accept passwords in an interactive fashion
from arcgis.features import FeatureLayer
from arcgis.features import SpatialDataFrame
from copy import deepcopy
import pandas as pd
import pickle
pd.set_option('display.max_columns', None)

/opt/anaconda3/lib/python3.7/site-packages/arcgis/features/_data/geodataset/utils.py:16: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime,


## Configure GIS connection

In [2]:
gis = GIS(url="http://lahub.maps.arcgis.com/home/organization.html",
          username="vivian.peng_lahub",password="#") #add your ArcGIS username and password here

## Update Test Positivity Layer

#### Load loayers from ArcGIS

In [3]:
# load test positivity layer
tests_layer = gis.content.search('title:Join_Test_Positivity_County_Zip')[1].layers[0] 
update_features = tests_layer.query().features

KeyboardInterrupt: 

#### Read in updated data

In [6]:
# read in updated data
test_positivity = pd.read_csv("data/test_positivity_zip.csv")
test_positivity["Zip"] = test_positivity["Zip"].astype(str)

In [7]:
test_positivity.head()

,Zip,Need New Sample,Negative,Positive,Total,Test Posivitiy,Date
0,0,0,1,0,1,0.0,2021-06-18
1,924,0,1,0,1,0.0,2021-06-18
2,982,0,1,0,1,0.0,2021-06-18
3,1056,0,1,0,1,0.0,2021-06-18
4,1830,0,1,0,1,0.0,2021-06-18


In [8]:
# Read in list of zip codes
with open ("data/LA_county_zips.data", "rb") as filehandle:
    LA_county_zips = pickle.load(filehandle)

In [9]:
# filter for LA County
test_positivity = test_positivity[test_positivity["Zip"].isin(LA_county_zips)]

In [10]:
#loop through ZIPs and set new values
for v in test_positivity.values:
    #[f for f in update_features if f.attributes['GEOID10']==v[0]][0].attributes["Indeterminate"]= v[1]
    [f for f in update_features if f.attributes['GEOID10']==v[0]][0].attributes["Need New Sample"]= v[1]
    [f for f in update_features if f.attributes['GEOID10']==v[0]][0].attributes['Negative'] = v[2]
    [f for f in update_features if f.attributes['GEOID10']==v[0]][0].attributes['Positive'] = v[3]
    [f for f in update_features if f.attributes['GEOID10']==v[0]][0].attributes['Total'] = v[4]
    [f for f in update_features if f.attributes['GEOID10']==v[0]][0].attributes['Test Posivitiy'] = v[5]
    [f for f in update_features if f.attributes['GEOID10']==v[0]][0].attributes['Date'] = v[6]

In [11]:
#edit features to match the updated features
for f in update_features:
    tests_layer.edit_features(updates=[f])

## Update Case/Deaths Layers

In [4]:
import numpy as np

In [5]:
# read in updated data
county = pd.read_csv("data/testing_neighborhoods.csv")
county = county[county["Neighborhood"]!='MISCELLANEOUS**']
county["Neighborhood"] = county["Neighborhood"].str.upper()

In [6]:
# clean vaccination rate columns
vax_columns = ["percent_of_ppl_vaccinated", "percent_12_to_17_vaccinated", "percent_65_plus_vaccinated"]

In [7]:
def clean_vax_columns(df, col):
    df[col] = df[col].replace("Unreliable Data", np.NaN)
    df[col] = df[col].str.replace("%", "")  
    df[col] = df[col].astype(float)

In [8]:
for col in vax_columns:
    clean_vax_columns(county, col)
    

In [9]:
def update_cases_deaths_layer(title, county):
    # load layers
    layer = gis.content.search(title)[0].layers[0] 
    update_features = layer.query().features
    hp_tb = county[(county["Neighborhood"]== "HYDE PARK") |(county["Neighborhood"]== "TEMPLE-BEAUDRY")]
    
    #loop through and set new values
    for v in county.values:
        [f for f in update_features if f.attributes['COMTY_NAME']==v[0]][0].attributes["New_Cases"]= v[4]
        [f for f in update_features if f.attributes['COMTY_NAME']==v[0]][0].attributes["New_Deaths"]= v[5]
        [f for f in update_features if f.attributes['COMTY_NAME']==v[0]][0].attributes['Two_Week_Crude_IR__per_100k_'] = v[1]
        [f for f in update_features if f.attributes['COMTY_NAME']==v[0]][0].attributes['adj_case_rate'] = v[2]
        [f for f in update_features if f.attributes['COMTY_NAME']==v[0]][0].attributes['Cumulative_Confirmed_Case_Count'] = v[7]
        [f for f in update_features if f.attributes['COMTY_NAME']==v[0]][0].attributes['Cumulative_Death_Count'] = v[8]
        [f for f in update_features if f.attributes['COMTY_NAME']==v[0]][0].attributes['Date_1'] = v[3]
        [f for f in update_features if f.attributes['COMTY_NAME']==v[0]][0].attributes['top_trends_cases'] = v[15]
        [f for f in update_features if f.attributes['COMTY_NAME']==v[0]][0].attributes['top_new_cases'] = v[16]
        [f for f in update_features if f.attributes['COMTY_NAME']==v[0]][0].attributes['vax_rate'] = v[11]
        [f for f in update_features if f.attributes['COMTY_NAME']==v[0]][0].attributes["persons_vaccinated"] = v[10]
        [f for f in update_features if f.attributes['COMTY_NAME']==v[0]][0].attributes['youth_vaccinated'] = v[12]
        [f for f in update_features if f.attributes['COMTY_NAME']==v[0]][0].attributes['seniors_vaccinated'] = v[13]
        
    #edit features to match the updated features
    for f in update_features:
        layer.edit_features(updates=[f])
    
    # Update Hyde Park
    hp_feature = [f for f in update_features if f.attributes['OBJECTID'] == 75][0]
    hp_edit = hp_feature
    hp_edit.attributes["New_Cases"]= hp_tb.iloc[0,4]
    hp_edit.attributes["New_Deaths"]= hp_tb.iloc[0,5]
    hp_edit.attributes['Two_Week_Crude_IR__per_100k_'] = hp_tb.iloc[0,1]
    hp_edit.attributes['adj_case_rate'] = hp_tb.iloc[0,2]
    hp_edit.attributes['Cumulative_Confirmed_Case_Count'] = hp_tb.iloc[0,7]
    hp_edit.attributes['Cumulative_Death_Count'] = hp_tb.iloc[0,8]
    hp_edit.attributes['Date_1'] = hp_tb.iloc[0,3]
    hp_edit.attributes['top_trends_cases'] = hp_tb.iloc[0,15]
    hp_edit.attributes['top_new_cases'] = hp_tb.iloc[0,16]
    hp_edit.attributes['vax_rate'] = hp_tb.iloc[0,11]
    hp_edit.attributes['persons_vaccinated'] = hp_tb.iloc[0,10]
    hp_edit.attributes['youth_vaccinated'] = hp_tb.iloc[0,12]
    hp_edit.attributes['seniors_vaccinated'] = hp_tb.iloc[0,13]
        
    # Update Temple-Beaudry  
    tb_feature = [f for f in update_features if f.attributes['OBJECTID'] == 57][0]
    tb_edit = tb_feature
    tb_edit.attributes["New_Cases"]= hp_tb.iloc[1,4]
    tb_edit.attributes["New_Deaths"]= hp_tb.iloc[1,5]
    tb_edit.attributes['Two_Week_Crude_IR__per_100k_'] = hp_tb.iloc[1,1]
    tb_edit.attributes['adj_case_rate'] = hp_tb.iloc[1,2]
    tb_edit.attributes['Cumulative_Confirmed_Case_Count'] = hp_tb.iloc[1,7]
    tb_edit.attributes['Cumulative_Death_Count'] = hp_tb.iloc[1,8]
    tb_edit.attributes['Date_1'] = hp_tb.iloc[1,3]
    tb_edit.attributes['top_trends_cases'] = hp_tb.iloc[1,15]
    tb_edit.attributes['top_new_cases'] = hp_tb.iloc[1,16]
    tb_edit.attributes['vax_rate'] = hp_tb.iloc[0,11]
    tb_edit.attributes['persons_vaccinated'] = hp_tb.iloc[0,10]
    tb_edit.attributes['youth_vaccinated'] = hp_tb.iloc[0,12]
    tb_edit.attributes['seniors_vaccinated'] = hp_tb.iloc[0,13]
    
    layer.edit_features(updates = [hp_edit])
    layer.edit_features(updates = [tb_edit])
    
    


In [9]:
# layers_to_update =["title:Two_Week_Case_Rates", "title:New_Cases_in_Last_7_Days", "title:New_Deaths_in_Last_7_Days","title:Top_Trends_Case_Rate",  "title:Top_Trends_New_Cases",]
# for layer in layers_to_update:
   # update_cases_deaths_layer(layer, county)

In [12]:
update_cases_deaths_layer('title:Two_Week_Case_Rates', county)

## Update Testing Sites Layer

In [13]:
from arcgis.features import FeatureLayerCollection

In [14]:
flayer = gis.content.get("1624e8fb02e54d138e2b5206dac23998")

In [15]:
flayer_collection = FeatureLayerCollection.fromitem(flayer)

In [16]:
flayer_collection.manager.overwrite("data/mobile_testing_sites.csv")

{'success': True}

## Creating new fields

In [5]:
layer = gis.content.search('title:Two_Week_Case_Rates')[0].layers[0] 
layer_fields = layer.manager.properties.fields

In [6]:
new_fields = ['youth_vaccinated', "seniors_vaccinated"]

In [22]:
template_field = dict(deepcopy(layer_fields[13]))

In [23]:
template_field

{'name': 'vax_rate',
 'type': 'esriFieldTypeDouble',
 'actualType': 'decimal',
 'alias': 'vax_rate',
 'sqlType': 'sqlTypeDecimal',
 'nullable': True,
 'editable': True,
 'visible': True,
 'domain': None,
 'defaultValue': None}

In [24]:
#set up new definition and add to dictionary
fields_to_be_added = []

#create ammended dictionary for each new field
for new_field_name in new_fields:
    current_field = deepcopy(template_field)
    current_field['name'] = new_field_name.lower()
    current_field['alias'] = new_field_name
    fields_to_be_added.append(current_field)
    
    
#add definition
layer.manager.add_to_definition({'fields':fields_to_be_added})

{'success': True}